Para el desarrollo de este proyecto, se ha seleccionado como fuente de información primaria el archivo SP1.csv (disponible en https://www.football-data.co.uk/spainm.php), el cual contiene el registro detallado de los partidos correspondientes a la Primera División de España (La Liga) para la temporada 2024-2025.

Este conjunto de datos nos proporciona una visión de cada encuentro disputado hasta la fecha. Gracias a la documentación técnica consultada (https://www.football-data.co.uk/notes.txt), identificamos que el archivo no solo se limita a los resultados deportivos (goles, victorias/derrotas), sino que estructura la información en tres grandes bloques: datos del encuentro (fecha, equipos), estadísticas de rendimiento (disparos, córners, faltas, tarjetas) y datos del mercado de apuestas (cuotas de apertura y cierre de múltiples casas de apuestas)

A partir de este conjunto de datos en bruto, procederemos a aplicar las técnicas de preprocesamiento y limpieza necesarias para garantizar la calidad de la información antes de abordar su visualización.

# Documentación Técnica: Pipeline ETL La Liga 2024-25

## Desglose Detallado del Algoritmo (Paso a Paso)

### PASO 1: Ingesta de la información primaria (SP1.csv) y Normalización Semántica

**¿Qué hace el código?**
Carga el archivo `SP1.csv` (estadísticas puras del juego). Pero el problema principal es la **heterogeneidad semántica**.

* **Dificultad Encontrada:** Las fuentes de datos no utilizan los mismos nombres de los diferentes equipos:
    * El CSV llama al equipo `At Bilbao`.
    * Wikipedia lo llama `Athletic Club`.
    * EstadiosDB lo llama `Athletic Club de Bilbao`.
* **Solución Técnica:** Se implementa un **Diccionario de Mapeo (`mapa_nombres`)** creado a partir de todos los nombres de los diferentes equipos que se han utilizado en las distintas fuentes de datos.  Antes de hacer cualquier cruce, obligamos a todos los DataFrames a usar una nomenclatura estándar.
    * *Lógica:* `df.replace(diccionario)` escanea todas las columnas de nombres y unifica las variantes.

---

### PASO 2: Web Scrapping sobre Wikipedia 1 (PARTIDOS, ASISTENCIA Y ESTADIOS). 

`SP1.csv` no posee la información de en qué estadio se jugó el cada partido ni de los datos de asistencia del mismo. Siendo asi, iteramos sobre 20 URLs (una por equipo) sobre su página en Wikipedia (única página que hemos encontrado que nos daba esta información) para obtener esta información.

**Mecánica Interna:**
1.  **Petición:** Descargamos el HTML de la temporada 24/25 de cada equipo.
2.  **Búsqueda del Contenedor:** Buscamos etiquetas `div` con la clase `vevent`. En el HTML de Wikipedia, `vevent` es un microformato estándar para "Eventos" (partidos).
3. **Dificultad A**: Filtrado de Contexto (Ruido en los Datos)
    * **El Problema**: Las páginas de Wikipedia mezclan partidos de La Liga, Copa del Rey y Amistosos en una misma lista visual. 
    * **Solución Técnica**: Búsqueda Inversa en el DOM (`find_previous`). Antes de aceptar un partido, el script mira "hacia arriba" en el código HTML buscando el encabezado (`<h2>` o `<h3>`) anterior más cercano. Solo si ese encabezado contiene el texto "La Liga", el script procesa el dato.
4. **Dificultad B**: Extracción de Datos "Sucios"
    * **El Problema**: El dato de asistencia en el HTML viene sucio, mezclado con texto y referencias bibliográficas. Ejemplo: "Attendance: 45,000 [3]". Python no puede sumar eso.
    * **Solución Técnica**: Usamos `re.search(r'Attendance:\s*([\d,]+)', ...)`. Esta fórmula ignora la palabra "Attendance", los espacios y los corchetes [3], extrayendo únicamente los dígitos numéricos para convertirlos a enteros (int).
5. **Dificultad C**: Estadios Vacíos
    * **El Problema**: A veces Wikipedia no escribe el nombre del estadio en la celda correspondiente si es el estadio habitual del equipo local, dejándolo vacío o como "Desconocido".
    * **Solución Técnica**: Aprendizaje y Relleno. Creamos un diccionario dinámico durante la ejecución. Si el script lee una fila completa donde dice "Local: Celta -> Estadio: Balaídos", guarda ese conocimiento. Más tarde, si encuentra "Local: Celta -> Estadio: Desconocido", usa el conocimiento previo para rellenar el hueco automáticamente.

---

### PASO 3: Web Scraping EstadiosDB (Asistencia Media)

Al no ser Wikipedia una fuente oficial de información, hay partidos en los cuales no se dispone de la asistencia de este. Necesitamos por tanto un validador externo, en este caso EstadiosDB, el cual tiene una tabla resumen de la asistencia media a cada estadio durante la temporada 24-25.

**¿Qué hace el código?**
Va a una URL específica de EstadiosDB que contiene una tabla resumen (`<table class="arttab">`).

* **Dificultad de Formato:** Los números en Europa usan puntos para miles (45.000) y en Python/USA usan comas o nada. Además, a veces tienen asteriscos (45.000*).
* **Limpieza:** El script aplica `.replace(".", "").replace("*", "")` para convertir el texto sucio en un número entero puro (`int`) que podamos sumar y restar matemáticamente.

---

### PASO 4: Imputación Estadística (Relleno de Huecos)

Aquí resolvemos el problema de los datos faltantes en la asistencia de los partidos.

**La Lógica Matemática:**
No usamos la media simple (que se vería afectada por los ceros), sino una **proyección inversa**.

1.  **Recuperamos la Media Oficial (M):** Del Paso 3 (ej: 50.000 espectadores).
2.  **Calculamos el Volumen Total Esperado (V_tot):**
    > `V_tot = Media Oficial * Número Total de Partidos Jugados`
3.  **Sumamos la Asistencia Real (V_real):** Suma de los partidos que SÍ tienen dato.
4.  **Calculamos el Déficit (Delta):**
    > `Delta = V_tot - V_real`
5.  **Reparto:** Ese `Delta` (los espectadores que "faltan" para cuadrar la media) se divide equitativamente entre los partidos que tienen 0 asistencia.

**Resultado:** Obtenemos un dataset completo donde la suma total es coherente con la realidad oficial del club.

---

### PASO 5: Geolocalización (Extracción de Coordenadas)

Necesitamos colocar los estadios en el mapa. Para ello, vamos a las páginas individuales de Wikipedia de cada estadio (ej: *Santiago Bernabéu*).

**Mecánica de Scraping:**
1.  **Detección de Nombre Oficial:** A veces el nombre coloquial ("Bernabéu") no es el oficial. Buscamos en la `infobox` (la tabla gris a la derecha en Wikipedia) el campo "Nombre completo".
2.  **Extracción Geoespacial:**
    * Wikipedia oculta las coordenadas para máquinas dentro de una etiqueta `<span class="geo">`.
    * *Ejemplo de HTML:* `<span class="geo">40.453; -3.688</span>`
    * *Dificultad:* A veces usan punto y coma (`;`) como separador, y a veces coma (`,`).
    * *Solución:* El código tiene un `if/else` para detectar qué separador se está usando, hace un `split`, y asigna Latitud (`[0]`) y Longitud (`[1]`).

---

### PASO 6: Guardado y Estructura Final

El script finaliza volcando la memoria RAM al disco duro. Se generan 3 archivos CSV clave para garantizar la integridad referencial:

1.  **`SP1_Normalizado.csv`**: La base de datos deportiva limpia.
2.  **`datos_partidos_asistencia.csv`**: La tabla de hechos (Fact Table) con el calendario y el público.
3.  **`datos_coordenadas.csv`**: La tabla de dimensiones (Dim Table) con la ubicación física.

---

## Resumen de Dificultades Técnicas Superadas

| Dificultad | Solución Aplicada en Código |
| :--- | :--- |
| **Bloqueo de Bots** | Inyección de cabeceras `User-Agent` falsas. |
| **Nombres Dispares** | Diccionarios maestros de normalización (`mapa_nombres`). |
| **Datos Sucios** | Limpieza con Regex y `.replace` (quitar comas, asteriscos, citas). |
| **Estadios Vacíos** | Lógica de inferencia basada en el equipo Local. |
| **Ceros en Asistencia** | Algoritmo matemático de imputación basado en la media oficial. |
| **Coordenadas Raras** | Parsers condicionales para distintos formatos de separadores GPS. |

In [3]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 2.8 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 523.7/523.7 kB 10.4 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 3.9 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# ==============================================================================
# PROYECTO: EXTRACCIÓN Y LIMPIEZA DE DATOS DE LALIGA 2024-25
# DESCRIPCIÓN: Script maestro para unificar estadísticas, asistencia y geolocalización.
# ==============================================================================

# ==============================================================================
# 1. IMPORTACIÓN DE LIBRERÍAS
# ==============================================================================
# Importamos las herramientas necesarias:
# 'os': Para manejar rutas de archivos y crear carpetas en el sistema operativo.
# 're': Para usar expresiones regulares (búsqueda de patrones de texto complejos).
# 'time': Para pausar la ejecución (sleep) y no ser bloqueados por los servidores web.
# 'io': Para manejo de flujos de entrada/salida (aunque en este script se usa poco).
# 'pandas': La herramienta principal para manipular tablas de datos (DataFrames).
# 'requests' y 'urllib': Para realizar peticiones a páginas web (descargar el HTML).
# 'BeautifulSoup': Para leer ("parsear") y extraer información específica del HTML descargado.

import os
import re
import time
import io
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

print("============================================")
print("--- INICIO DEL PROCESO MAESTRO DE DATOS ---")
print("============================================")
print("[INFO] Cargando librerías y configuraciones iniciales...")

# ==============================================================================
# 2. CONFIGURACIÓN DEL ENTORNO Y RUTAS
# ==============================================================================
# Definimos dónde están los archivos y dónde se guardarán los resultados.

# Ruta donde se encuentra tu archivo original de estadísticas (CSV base).
RUTA_CSV_ESTADISTICAS = r"inputs/SP1.csv"

# Carpeta donde guardaremos todos los archivos generados.
CARPETA_SALIDA = r"outputs"
# Verificamos si la carpeta de salida existe; si no, la creamos automáticamente.
if not os.path.exists(CARPETA_SALIDA):
    os.makedirs(CARPETA_SALIDA)
    print(f"[INFO] Carpeta creada: {CARPETA_SALIDA}")

# Configuración de 'User-Agent':
# Esto es una "máscara" para que Wikipedia y otras webs crean que somos un navegador Chrome real
# y no un robot (script), evitando que nos bloqueen la conexión.
HEADERS_REQUEST = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# ==============================================================================
# 3. DICCIONARIOS MAESTROS DE NORMALIZACIÓN
# ==============================================================================
# Estos diccionarios actúan como traductores. Aseguran que "Real Madrid", "Real Madrid CF" 
# y "R. Madrid" sean siempre "Real Madrid". Esto es VITAL para poder cruzar las tablas después.

# MAPA DE EQUIPOS (Normalización de nombres)
mapa_nombres = {
    # DEPORTIVO ALAVÉS
    'Alaves': 'Deportivo Alavés', 
    'Alavés': 'Deportivo Alavés', 
    'Deportivo Alavés': 'Deportivo Alavés',
    # ATHLETIC CLUB
    'Ath Bilbao': 'Athletic Club', 
    'Athletic Bilbao': 'Athletic Club', 
    'Bilbao': 'Athletic Club', 
    'Athletic Club': 'Athletic Club',
    # ATLÉTICO DE MADRID
    'Ath Madrid': 'Atlético de Madrid', 
    'Atlético Madrid': 'Atlético de Madrid', 
    'Atlético de Madrid': 'Atlético de Madrid',
    # FC BARCELONA
    'Barcelona': 'FC Barcelona', 
    'FC Barcelona': 'FC Barcelona',
    # REAL BETIS
    'Betis': 'Real Betis', 
    'Real Betis': 'Real Betis',
    # RC CELTA
    'Celta': 'RC Celta', 
    'Celta Vigo': 'RC Celta', 
    'Celta de Vigo': 'RC Celta', 
    'RC Celta de Vigo': 'RC Celta', 
    'RC Celta': 'RC Celta',
    # RCD ESPANYOL
    'Espanol': 'RCD Espanyol', 
    'Espanyol': 'RCD Espanyol', 
    'RCD Espanyol': 'RCD Espanyol',
    # GETAFE CF
    'Getafe': 'Getafe CF', 
    'Getafe CF': 'Getafe CF',
    # GIRONA FC
    'Girona': 'Girona FC', 
    'Girona FC': 'Girona FC',
    # UD LAS PALMAS
    'Las Palmas': 'UD Las Palmas', 
    'UD Las Palmas': 'UD Las Palmas',
    # CD LEGANÉS
    'Leganes': 'CD Leganés', 
    'Leganés': 'CD Leganés', 
    'CD Leganés': 'CD Leganés',
    # RCD MALLORCA
    'Mallorca': 'RCD Mallorca', 
    'RCD Mallorca': 'RCD Mallorca',
    # CA OSASUNA
    'Osasuna': 'CA Osasuna', 
    'CA Osasuna': 'CA Osasuna',
    # RAYO VALLECANO
    'Vallecano': 'Rayo Vallecano', 
    'Rayo Vallecano': 'Rayo Vallecano',
    # REAL MADRID
    'Real Madrid': 'Real Madrid', 
    'Real Madrid CF': 'Real Madrid',
    # REAL SOCIEDAD
    'Sociedad': 'Real Sociedad', 
    'Real Sociedad': 'Real Sociedad',
    # SEVILLA FC
    'Sevilla': 'Sevilla FC', 
    'Sevilla FC': 'Sevilla FC',
    # VALENCIA CF
    'Valencia': 'Valencia CF', 
    'Valencia CF': 'Valencia CF',
    # REAL VALLADOLID
    'Valladolid': 'Real Valladolid', 
    'Real Valladolid': 'Real Valladolid',
    # VILLARREAL CF
    'Villarreal': 'Villarreal CF', 
    'Villarreal CF': 'Villarreal CF'
}

# MAPA DE ESTADIOS (Normalización de nombres de estadios)
mapa_estadios = {
    # DEPORTIVO ALAVÉS
    'Mendizorrotza': 'Estadio de Mendizorroza', 
    'Mendizorrotza Stadium': 'Estadio de Mendizorroza', 
    'Estadio de Mendizorroza': 'Estadio de Mendizorroza', 
    'Mendizorroza': 'Estadio de Mendizorroza',
    # ATHLETIC CLUB
    'San Mamés': 'Estadio de San Mamés', 
    'Estadio San Mamés': 'Estadio de San Mamés', 
    'Estadio de San Mamés': 'Estadio de San Mamés',
    # ATLÉTICO DE MADRID
    'Cívitas Metropolitano': 'Estadio Metropolitano', 
    'Riyadh Air Metropolitano': 'Estadio Metropolitano', 
    'Estadio Riyadh Air Metropolitano': 'Estadio Metropolitano', 
    'Metropolitano': 'Estadio Metropolitano', 
    'Metropolitano Stadium': 'Estadio Metropolitano', 
    'Estadio Metropolitano': 'Estadio Metropolitano',
    # FC BARCELONA
    'Estadi Olímpic Lluís Companys': 'Estadi Olímpic Lluís Companys', 
    'Lluís Companys Olympic Stadium': 'Estadi Olímpic Lluís Companys', 
    'Estadio Olímpico Lluís Companys': 'Estadi Olímpic Lluís Companys', 
    'Olímpic Lluís Companys': 'Estadi Olímpic Lluís Companys',
    # REAL BETIS
    'Benito Villamarín': 'Estadio Benito Villamarín', 
    'Estadio Benito Villamarín': 'Estadio Benito Villamarín',
    # RC CELTA
    'Balaídos': 'Estadio de Balaídos', 
    'Abanca-Balaídos': 'Estadio de Balaídos', 
    'Estadio Abanca Balaídos': 'Estadio de Balaídos', ''
    'Estadio de Balaídos': 'Estadio de Balaídos', 
    'ABANCA Balaídos': 'Estadio de Balaídos',
    # RCD ESPANYOL
    'RCDE Stadium': 'RCDE Stadium', 
    'Stage Front Stadium': 'RCDE Stadium',
    # GETAFE CF
    'Coliseum': 'Estadio Coliseum', 
    'Coliseum (Getafe)': 'Estadio Coliseum', 
    'Estadio Coliseum': 'Estadio Coliseum',
    # GIRONA FC
    'Montilivi': 'Estadi Montilivi', 
    'Estadi Montilivi': 'Estadi Montilivi', 
    'Estadio Municipal de Montilivi': 'Estadi Montilivi',
    # UD LAS PALMAS
    'Gran Canaria': 'Estadio de Gran Canaria', 
    'Estadio Gran Canaria': 'Estadio de Gran Canaria', 
    'Estadio de Gran Canaria': 'Estadio de Gran Canaria',
    # CD LEGANÉS
    'Butarque': 'Estadio Municipal de Butarque', 
    'Estadio Municipal Butarque': 'Estadio Municipal de Butarque', 
    'Estadio Municipal de Butarque': 'Estadio Municipal de Butarque', 
    'Municipal de Butarque': 'Estadio Municipal de Butarque',
    # RCD MALLORCA
    'Son Moix': 'Estadi Mallorca Son Moix', 
    'Mallorca Son Moix': 'Estadi Mallorca Son Moix', 
    'Estadi Mallorca Son Moix': 'Estadi Mallorca Son Moix', 
    'Estadio de Son Moix': 'Estadi Mallorca Son Moix', 
    'Visit Mallorca Estadi': 'Estadi Mallorca Son Moix',
    # CA OSASUNA
    'El Sadar': 'Estadio El Sadar', 
    'Estadio El Sadar': 'Estadio El Sadar',
    # RAYO VALLECANO
    'Vallecas': 'Estadio de Vallecas', 
    'Estadio de Vallecas': 'Estadio de Vallecas', 
    'Campo de Fútbol de Vallecas': 'Estadio de Vallecas', 
    'Vallecas Stadium': 'Estadio de Vallecas',
    # REAL MADRID
    'Santiago Bernabéu': 'Estadio Santiago Bernabéu', 
    'Estadio Santiago Bernabéu': 'Estadio Santiago Bernabéu', 
    'Santiago Bernabéu Stadium': 'Estadio Santiago Bernabéu',
    # REAL SOCIEDAD
    'Anoeta': 'Estadio de Anoeta', 
    'Anoeta Stadium': 'Estadio de Anoeta', 
    'Reale Arena': 'Estadio de Anoeta', 
    'Estadio de Anoeta': 'Estadio de Anoeta',
    # SEVILLA FC
    'Ramón Sánchez Pizjuán': 'Estadio Ramón Sánchez-Pizjuán', 
    'Estadio Ramón Sánchez-Pizjuán': 'Estadio Ramón Sánchez-Pizjuán', 
    'Ramón Sánchez Pizjuán Stadium': 'Estadio Ramón Sánchez-Pizjuán', 
    'Ramón Sánchez-Pizjuán': 'Estadio Ramón Sánchez-Pizjuán',
    # VALENCIA CF
    'Mestalla': 'Estadio de Mestalla', 
    'Estadio de Mestalla': 'Estadio de Mestalla',
    # REAL VALLADOLID
    'José Zorrilla': 'Estadio José Zorrilla', 
    'Estadio José Zorrilla': 'Estadio José Zorrilla',
    # VILLARREAL CF
    'La Cerámica': 'Estadio de la Cerámica', 
    'Estadio de la Cerámica': 'Estadio de la Cerámica',
    # GENÉRICOS
    'Desconocido': 'Desconocido'
}

# ==============================================================================
# PASO 1: CARGA Y NORMALIZACIÓN DEL ARCHIVO BASE (SP1.csv)
# ==============================================================================
print("\n-------------------------------------------------------------")
print("--> PASO 1: Cargando y Normalizando archivo base (SP1.csv)")
print("-------------------------------------------------------------")

if os.path.exists(RUTA_CSV_ESTADISTICAS):
    # Leemos el CSV
    df_base = pd.read_csv(RUTA_CSV_ESTADISTICAS)
    print(f"   [OK] Archivo cargado. Filas totales: {len(df_base)}")
    
    if 'HomeTeam' in df_base.columns and 'AwayTeam' in df_base.columns:
        # Normalizamos los nombres de los equipos en el archivo original
        print("   [...] Normalizando nombres de equipos en el CSV base...")
        df_base['HomeTeam'] = df_base['HomeTeam'].replace(mapa_nombres)
        df_base['AwayTeam'] = df_base['AwayTeam'].replace(mapa_nombres)
        
        # Guardamos una copia limpia
        ruta_sp1_norm = os.path.join(CARPETA_SALIDA, "SP1_Normalizado.csv")
        df_base.to_csv(ruta_sp1_norm, index=False, encoding='utf-8-sig')
        print(f"   [GUARDADO] Archivo 'SP1_Normalizado.csv' listo.")
    else:
        print("   [ALERTA] No se encontraron columnas 'HomeTeam'/'AwayTeam'.")
else:
    print(f"   [ERROR] No se encuentra el archivo en {RUTA_CSV_ESTADISTICAS}")
    df_base = pd.DataFrame() 

# ==============================================================================
# PASO 2: WEB SCRAPING DE WIKIPEDIA (PARTIDOS, ASISTENCIA Y ESTADIOS)
# ==============================================================================
print("\n-------------------------------------------------------------")
print("--> PASO 2: Extrayendo datos de PARTIDOS desde Wikipedia")
print("-------------------------------------------------------------")

# Lista de URLs de las temporadas 2024-25 para cada equipo
urls_partidos = [
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_Athletic_Bilbao_season", 
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_Atl%C3%A9tico_Madrid_season",
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_FC_Barcelona_season", 
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_RC_Celta_de_Vigo_season",
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_Deportivo_Alav%C3%A9s_season", 
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_RCD_Espanyol_season",
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_Getafe_CF_season", 
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_Girona_FC_season",
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_UD_Las_Palmas_season", 
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_CD_Legan%C3%A9s_season",
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_RCD_Mallorca_season", 
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_CA_Osasuna_season",
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_Rayo_Vallecano_season", 
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_Real_Betis_season",
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_Real_Madrid_CF_season", 
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_Real_Sociedad_season",
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_Sevilla_FC_season", 
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_Valencia_CF_season",
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_Real_Valladolid_season", 
    "https://en.wikipedia.org/wiki/2024%E2%80%9325_Villarreal_CF_season"
]

all_data_partidos = []

# Iteramos sobre cada URL para extraer los partidos
for i, url in enumerate(urls_partidos):
    # Mostramos progreso en pantalla para saber que no se ha colgado
    print(f"   > Procesando equipo {i+1}/{len(urls_partidos)}: {url.split('wiki/')[1]}")
    
    try:
        req = Request(url, headers=HEADERS_REQUEST)
        html = urlopen(req)
        soup = BeautifulSoup(html, "html.parser")
        
        # Buscamos las cajas de los partidos (clase 'vevent' en Wikipedia)
        matches = soup.find_all("div", {"class": "vevent"})
        
        for box in matches:
            try:
                # Verificamos que sea un partido de "La Liga" mirando el encabezado previo
                prev_header = box.find_previous(["h2", "h3"])
                if prev_header and "la liga" in prev_header.get_text().lower():
                    
                    teams_spans = box.find_all("span", class_="fn org")
                    if len(teams_spans) >= 2:
                        home = teams_spans[0].get_text(strip=True)
                        away = teams_spans[1].get_text(strip=True)
                        
                        # Extraemos estadio y asistencia
                        stadium = box.find("span", class_="location").get_text(strip=True) if box.find("span", class_="location") else "Desconocido"
                        
                        att_match = re.search(r'Attendance:\s*([\d,]+)', box.get_text())
                        attendance = int(att_match.group(1).replace(",", "")) if att_match else 0
                        
                        all_data_partidos.append([home, away, stadium, attendance])
            except: continue
    except: 
        print(f"   [ERROR] Fallo al leer URL: {url}")
        continue

# Creamos el DataFrame con todo lo extraído
df_wiki = pd.DataFrame(all_data_partidos, columns=["Local", "Visitante", "Estadio", "Asistencia"])

# --- NORMALIZACIÓN DE SCRAPING ---
print("   [...] Normalizando nombres extraídos de Wikipedia...")
df_wiki["Local"] = df_wiki["Local"].replace(mapa_nombres)
df_wiki["Visitante"] = df_wiki["Visitante"].replace(mapa_nombres)
df_wiki["Estadio"] = df_wiki["Estadio"].replace(mapa_estadios)

print(f"   [INFO] Total partidos brutos encontrados: {len(df_wiki)}")

# --- LIMPIEZA Y CORRECCIÓN DE ESTADIOS ---
print("   [...] Rellenando estadios 'Desconocido' basándonos en el equipo local...")

# Filtramos las filas que SÍ tienen estadio conocido
df_con_estadio = df_wiki[df_wiki['Estadio'] != 'Desconocido']

# Creamos un "mapa de aprendizaje": {Equipo Local -> Estadio Típico}
mapa_estadios_detectados = df_con_estadio.drop_duplicates(subset=['Local']).set_index('Local')['Estadio'].to_dict()

def rellenar_estadio(row):
    if row['Estadio'] == 'Desconocido':
        return mapa_estadios_detectados.get(row['Local'], 'Desconocido')
    return row['Estadio']

# Aplicamos la corrección
df_wiki['Estadio'] = df_wiki.apply(rellenar_estadio, axis=1)

# --- DEDUPLICACIÓN FINAL ---
# Si hay datos duplicados, nos quedamos con el que tenga mayor asistencia (dato más completo)
df_wiki['Asistencia'] = pd.to_numeric(df_wiki['Asistencia'], errors='coerce').fillna(0)
df_wiki['tiene_estadio'] = df_wiki['Estadio'] != "Desconocido"
df_wiki = df_wiki.sort_values(by=['Asistencia', 'tiene_estadio'], ascending=[False, False])
df_wiki = df_wiki.drop_duplicates(subset=['Local', 'Visitante'], keep='first')
df_wiki = df_wiki.drop(columns=['tiene_estadio']).reset_index(drop=True)

print(f"   [RESULTADO] Total partidos finales tras limpieza: {len(df_wiki)}")
if len(df_wiki) == 380:
    print("   [ÉXITO] Calendario completo (380 partidos).")
else:
    print(f"   [AVISO] Faltan partidos o el calendario aún no ha terminado ({len(df_wiki)} partidos).")

# ==============================================================================
# PASO 3: WEB SCRAPING ESTADIOSDB (ASISTENCIA MEDIA)
# ==============================================================================
print("\n-------------------------------------------------------------")
print("--> PASO 3: Extrayendo ASISTENCIA MEDIA de EstadiosDB")
print("-------------------------------------------------------------")

url_resumen = "https://estadiosdb.com/noticias/2025/06/espana_asistencia_a_los_estadios_de_la_liga_en_la_temporada_202425"
data_resumen = []

try:
    print(f"   > Conectando a: {url_resumen}")
    resp = requests.get(url_resumen, headers=HEADERS_REQUEST)
    soup = BeautifulSoup(resp.text, "html.parser")
    table = soup.find("table", class_="arttab")
    
    if table:
        rows = table.find_all("tr")[1:] # Saltamos la cabecera
        for row in rows:
            cols = row.find_all("td")
            if len(cols) >= 6:
                club = cols[1].get_text(strip=True)
                est = cols[2].get_text(strip=True)
                # Limpiamos puntos y asteriscos de los números
                cap = int(cols[3].get_text(strip=True).replace(".", "").replace("*", ""))
                asis_media = int(cols[4].get_text(strip=True).replace(".", "").replace("*", ""))
                
                data_resumen.append([club, est, asis_media])
        print("   [OK] Datos de EstadiosDB extraídos correctamente.")
    else:
        print("   [ERROR] No se encontró la tabla en EstadiosDB.")

except Exception as e:
    print(f"   [ERROR CRÍTICO] EstadiosDB: {e}")

df_estadios = pd.DataFrame(data_resumen, columns=["Club", "Estadio_DB", "Asistencia_Media"])
df_estadios["Club"] = df_estadios["Club"].replace(mapa_nombres)

# ==============================================================================
# PASO 4: IMPUTACIÓN (RELLENO INTELIGENTE) DE DATOS FALTANTES
# ==============================================================================
print("\n-------------------------------------------------------------")
print("--> PASO 4: Rellenando huecos de asistencia (< 1000 espectadores)")
print("-------------------------------------------------------------")
# Lógica: Si faltan datos de asistencia en un partido, usamos la media del equipo 
# ajustada para que la suma total coincida con la realidad aproximada.

def es_dato_valido(valor):
    return valor >= 1000 

count_correcciones = 0

for index, row in df_estadios.iterrows():
    equipo = row['Club']
    media_temporada = row['Asistencia_Media']
    
    mask_local = df_wiki['Local'] == equipo
    partidos_local = df_wiki[mask_local]
    
    total_partidos = len(partidos_local)
    if total_partidos == 0: continue

    # Separamos partidos con dato bien vs partidos con dato mal (0 o muy bajo)
    partidos_validos = partidos_local[partidos_local['Asistencia'].apply(es_dato_valido)]
    partidos_a_corregir = partidos_local[~partidos_local['Asistencia'].apply(es_dato_valido)]
    
    num_a_corregir = len(partidos_a_corregir)
    if num_a_corregir == 0: continue
        
    # Calculamos cuánto falta para llegar a la media teórica
    asistencia_total_teorica = media_temporada * total_partidos
    asistencia_real_acumulada = partidos_validos['Asistencia'].sum()
    asistencia_faltante_total = asistencia_total_teorica - asistencia_real_acumulada
    
    if asistencia_faltante_total > 0:
        nueva_estimacion = int(asistencia_faltante_total / num_a_corregir)
    else:
        nueva_estimacion = int(media_temporada)

    print(f"   > Corrección en {equipo}: {num_a_corregir} partidos imputados con valor {nueva_estimacion}")
    
    indices_a_corregir = partidos_a_corregir.index
    df_wiki.loc[indices_a_corregir, 'Asistencia'] = nueva_estimacion
    count_correcciones += num_a_corregir

print(f"   [FIN] Total de partidos corregidos: {count_correcciones}")

# ==============================================================================
# PASO 5: EXTRACCIÓN DE COORDENADAS Y NOMBRES OFICIALES
# ==============================================================================
print("\n-------------------------------------------------------------")
print("--> PASO 5: Obteniendo Latitud/Longitud y Nombres Oficiales")
print("-------------------------------------------------------------")

urls_estadios = [
    "https://es.wikipedia.org/wiki/Estadio_de_Mendizorroza",
    "https://es.wikipedia.org/wiki/Estadio_de_San_Mam%C3%A9s",
    "https://es.wikipedia.org/wiki/Estadio_Metropolitano_(Madrid)",
    "https://es.wikipedia.org/wiki/Estadio_Ol%C3%ADmpico_Llu%C3%ADs_Companys",
    "https://es.wikipedia.org/wiki/Estadio_de_Bala%C3%ADdos",
    "https://es.wikipedia.org/wiki/RCDE_Stadium",
    "https://es.wikipedia.org/wiki/Coliseum_(Getafe)",
    "https://es.wikipedia.org/wiki/Estadio_Municipal_de_Montilivi",
    "https://es.wikipedia.org/wiki/Estadio_de_Gran_Canaria",
    "https://es.wikipedia.org/wiki/Estadio_Municipal_de_Butarque",
    "https://es.wikipedia.org/wiki/Estadio_de_Son_Moix",
    "https://es.wikipedia.org/wiki/Estadio_El_Sadar",
    "https://es.wikipedia.org/wiki/Estadio_de_Vallecas",
    "https://es.wikipedia.org/wiki/Estadio_Benito_Villamar%C3%ADn",
    "https://es.wikipedia.org/wiki/Estadio_Santiago_Bernab%C3%A9u",
    "https://es.wikipedia.org/wiki/Estadio_de_Anoeta",
    "https://es.wikipedia.org/wiki/Estadio_Jos%C3%A9_Zorrilla",
    "https://es.wikipedia.org/wiki/Estadio_Ram%C3%B3n_S%C3%A1nchez-Pizju%C3%A1n",
    "https://es.wikipedia.org/wiki/Estadio_de_Mestalla",
    "https://es.wikipedia.org/wiki/Estadio_de_la_Cer%C3%A1mica"
]

data_geo = []

for i, url in enumerate(urls_estadios):
    # Print de progreso para la geolocalización
    nombre_temp = url.split('/')[-1].replace("_", " ")
    print(f"   > Geolocalizando estadio {i+1}/{len(urls_estadios)}: {nombre_temp}...")
    
    try:
        response = requests.get(url, headers=HEADERS_REQUEST)
        if response.status_code != 200:
            print(f"     [ERROR HTTP] Código {response.status_code}")
            continue

        soup = BeautifulSoup(response.text, "html.parser")
        
        # --- EXTRACCIÓN DEL NOMBRE OFICIAL ---
        nombre_final = "Desconocido"
        infobox = soup.find("table", class_="infobox")
        
        encontrado_en_tabla = False
        if infobox:
            rows = infobox.find_all("tr")
            for row in rows:
                header = row.find("th")
                if header and "Nombre completo" in header.get_text(strip=True):
                    td = row.find("td")
                    if td:
                        nombre_final = td.get_text(strip=True)
                        encontrado_en_tabla = True
                        break
        
        if not encontrado_en_tabla:
            h1 = soup.find("h1", id="firstHeading")
            if h1:
                nombre_final = h1.get_text(strip=True)

        nombre_final = re.sub(r'\[.*?\]', '', nombre_final).strip()

        # --- EXTRACCIÓN DE COORDENADAS (GEO) ---
        latitud = None
        longitud = None
        
        geo_span = soup.find("span", class_="geo")
        
        if geo_span:
            coord_text = geo_span.get_text(strip=True)
            # Manejo de formatos con ";" o con ","
            if ";" in coord_text:
                 coords = coord_text.split(";") 
            else:
                 coords = coord_text.split(",")
            
            if len(coords) >= 2:
                latitud = coords[0].strip()
                longitud = coords[1].strip()
        
        data_geo.append([nombre_final, latitud, longitud])
        
    except Exception as e:
        print(f"     [EXCEPCIÓN] Error procesando {url}: {e}")
    
    # Pequeña pausa para ser "educados" con el servidor
    time.sleep(0.5)

df_geo = pd.DataFrame(data_geo, columns=["Estadio_Oficial", "Latitud", "Longitud"])
df_geo["Estadio_Oficial"] = df_geo["Estadio_Oficial"].replace(mapa_estadios)

# ==============================================================================
# PASO 6: GUARDADO DE TODOS LOS ARCHIVOS FINALES
# ==============================================================================
print("\n-------------------------------------------------------------")
print("--> PASO 6: Guardando resultados en disco")
print("-------------------------------------------------------------")

# 1. Archivo de Coordenadas
ruta_geo = os.path.join(CARPETA_SALIDA, "datos_coordenadas.csv")
df_geo.to_csv(ruta_geo, index=False, encoding='utf-8-sig')

# 2. Archivo de Partidos y Asistencia (Corregido y con Estadios Rellenados)
ruta_partidos = os.path.join(CARPETA_SALIDA, "datos_partidos_asistencia.csv")
df_wiki.to_csv(ruta_partidos, index=False, encoding='utf-8-sig')

# 3. Archivo de Capacidad y Asistencia Media (EstadiosDB)
ruta_estadios_media = os.path.join(CARPETA_SALIDA, "datos_asistencia_media_estadios.csv")
df_estadios.to_csv(ruta_estadios_media, index=False, encoding='utf-8-sig')

print(f"✅ PROCESO COMPLETADO EXITOSAMENTE.")
print(f"📂 Archivos generados en: {CARPETA_SALIDA}")
print("   1. SP1_Normalizado.csv")
print("   2. datos_coordenadas.csv")
print("   3. datos_partidos_asistencia.csv")
print("   4. datos_asistencia_media_estadios.csv")
print("-------------------------------------------------------------")

--- INICIO DEL PROCESO MAESTRO DE DATOS ---
[INFO] Cargando librerías y configuraciones iniciales...

-------------------------------------------------------------
--> PASO 1: Cargando y Normalizando archivo base (SP1.csv)
-------------------------------------------------------------
   [OK] Archivo cargado. Filas totales: 380
   [...] Normalizando nombres de equipos en el CSV base...
   [GUARDADO] Archivo 'SP1_Normalizado.csv' listo.

-------------------------------------------------------------
--> PASO 2: Extrayendo datos de PARTIDOS desde Wikipedia
-------------------------------------------------------------
   > Procesando equipo 1/20: 2024%E2%80%9325_Athletic_Bilbao_season
   [ERROR] Fallo al leer URL: https://en.wikipedia.org/wiki/2024%E2%80%9325_Athletic_Bilbao_season
   > Procesando equipo 2/20: 2024%E2%80%9325_Atl%C3%A9tico_Madrid_season
   [ERROR] Fallo al leer URL: https://en.wikipedia.org/wiki/2024%E2%80%9325_Atl%C3%A9tico_Madrid_season
   > Procesando equipo 3/20: 2024%E

# 🌦️ Documentación Técnica: Enriquecimiento Climático (Open-Meteo)

## 📋 1. Introducción y Objetivo

El objetivo de este script es añadir una **Dimensión Ambiental** al dataset maestro de partidos. No basta con saber *quién* ganó o *cuánta* gente fue; queremos saber *en qué condiciones* se jugó.

El script consulta una API meteorológica histórica para obtener las condiciones exactas (temperatura, lluvia, viento) en las coordenadas precisas del estadio y a la hora exacta del pitido inicial.

**¿Por qué es importante?**
Permite responder preguntas de negocio como:
* *"¿La lluvia reduce drásticamente la asistencia en estadios descubiertos?"*
* *"¿Afecta el calor extremo (>30°C) al número de goles marcados?"*
* *"¿Hay una correlación entre sensación térmica baja y venta de entradas?"*

---

## ⚙️ 2. Arquitectura del Script

El flujo de trabajo es lineal y robusto, diseñado para manejar fallos de red y límites de API.

1.  **Configuración del Cliente API:** Setup de caché y reintentos (Retries).
2.  **Ingesta de Datos:** Carga del CSV con coordenadas geográficas (`Latitud`, `Longitud`) y temporales (`Date`, `Time`).
3.  **Iteración y Consulta:** Bucle `for` que procesa cada partido individualmente.
4.  **Extracción de Precisión:** Filtrado del dato horario exacto del evento.
5.  **Consolidación:** Unión de las nuevas métricas al DataFrame original.

---

## 🔍 3. Análisis de la Fuente de Datos: Open-Meteo

Utilizamos la **Historical Weather API** de Open-Meteo. A diferencia de otras APIs, esta no requiere *API Key* para uso no comercial, pero requiere un manejo cuidadoso de las peticiones para no ser bloqueados.

### Parámetros de la Petición (Payload)

Para cada partido, el script construye una petición dinámica con estos parámetros:

| Parámetro | Valor Ejemplo | Descripción |
| :--- | :--- | :--- |
| `latitude` / `longitude` | `40.416`, `-3.703` | Ubicación exacta del estadio. |
| `start_date` / `end_date` | `2024-09-15` | Limitamos la búsqueda a un solo día. |
| `hourly` | `temperature_2m` | Temperatura a 2 metros del suelo. |
| `hourly` | `apparent_temperature` | Sensación térmica (Heat Index / Wind Chill). |
| `hourly` | `precipitation` | Suma de lluvia + nieve en mm. |
| `hourly` | `wind_speed_10m` | Velocidad del viento a 10 metros de altura. |
| `hourly` | `weather_code` | Código WMO (Numérico) que resume el clima. |
| `timezone` | `Europe/Madrid` | **Crítico:** Asegura que las 18:00 sean hora local, no UTC. |

---

## 🛠️ 4. Lógica Detallada paso a paso

### 🟢 PASO 1: Configuración de Robustez (Caché y Reintentos)

Las peticiones HTTP pueden fallar por mil razones (micro-cortes de internet, servidor ocupado). Para evitar que el script se rompa a la mitad (ej: en el partido 200 de 380), implementamos dos capas de seguridad:

1.  **`requests_cache`:**
    * Crea un archivo oculto `.cache`.
    * *Función:* Si ejecutas el script dos veces, la segunda vez **no conecta a internet** para los datos que ya tiene. Los lee del disco. Esto ahorra tiempo y evita baneos.
2.  **`retry_requests`:**
    * Configurado con `retries=5` y `backoff_factor=0.2`.
    * *Función:* Si la API da error 500 o timeout, el script espera 0.2 segundos y reintenta. Si falla, espera 0.4s, luego 0.8s... así hasta 5 veces antes de rendirse.

### 🟡 PASO 2: Procesamiento Temporal (La fecha ISO)

El CSV original suele tener fechas en formato humano/europeo (`DD/MM/YYYY`, ej: `15/09/2024`), pero las APIs siempre exigen el estándar internacional **ISO 8601** (`YYYY-MM-DD`).

* **Transformación:**
    ```python
    day, month, year = date_raw.split('/')
    api_date = f"{year}-{month}-{day}" # Resultado: "2024-09-15"
    ```

### 🟠 PASO 3: Extracción Horaria (El "Francotirador")

La API de Open-Meteo devuelve un array con **24 valores** (uno por cada hora del día solicitado: 00:00, 01:00... 23:00).

* **El Reto:** No queremos la temperatura media del día, queremos la temperatura a la hora del partido.
* **La Lógica:**
    1.  Tomamos la hora del partido: `18:30`.
    2.  Extraemos la hora entera: `18` (variable `match_hour`).
    3.  Usamos ese número como **índice** para acceder al array de la API.
    * *Ejemplo:* `response.Hourly().Variables(0).ValuesAsNumpy()[18]` nos da la temperatura exacta a las 18:00.

### 🔴 PASO 4: Manejo de Errores (Try/Except)

Es posible que algunos partidos tengan datos erróneos (ej: coordenadas vacías, fechas futuras que la API histórica no tiene).

* **Bloque `try...except`:**
    * Si ocurre un error en una fila específica, el script **no se detiene**.
    * Imprime un aviso en consola (`[ERROR] Fallo en fila X...`).
    * Rellena los campos climáticos con `None` (Vacío) y salta al siguiente partido.

---

## 📂 5. Resultados y Estructura de Salida

El archivo generado, `partidos_con_clima_completo.csv`, mantiene todas las columnas originales y añade 5 nuevas métricas al final:

| Nueva Columna | Unidad | Significado |
| :--- | :--- | :--- |
| **`Temperatura_C`** | Grados Celsius (°C) | Temperatura real del aire. |
| **`Sensacion_Termica_C`** | Grados Celsius (°C) | Cómo se siente el cuerpo humano (humedad/viento). |
| **`Precipitacion_mm`** | Milímetros (mm) | Cantidad de lluvia en esa hora (0 = Seco). |
| **`Viento_kmh`** | km/h | Velocidad media del viento. |
| **`Codigo_Clima`** | WMO Code (0-99) | [Tabla de códigos WMO](https://www.nodc.noaa.gov/archive/arc0021/0002199/1.1/data/0-data/HTML/WMO-CODE/WMO4677.HTM). <br>Ej: `0`=Despejado, `61`=Lluvia ligera, `95`=Tormenta. |

---

## ⚠️ Dificultades Técnicas Superadas

| Dificultad | Solución Técnica |
| :--- | :--- |
| **Límites de API** | Uso de caché local para evitar peticiones redundantes. |
| **Inestabilidad de Red** | Implementación de sistema de *Retries* (reintentos exponenciales). |
| **Formato de Fecha** | Conversión manual de string `DD/MM/YYYY` a `YYYY-MM-DD`. |
| **Precisión Horaria** | Indexación directa del array Numpy devuelto por la API usando la hora del evento. |
| **Datos Faltantes** | Verificación previa de coordenadas (`pd.isna`) antes de llamar a la API. |
| **Bloqueo SSL** | Uso de `urllib3.disable_warnings` para redes corporativas o Wi-Fi restringido. |

---

## ✅ Conclusión

Este script transforma un dataset estático en uno dinámico y contextual. Al cruzar coordenadas espaciales y temporales con bases de datos meteorológicas, habilitamos la capacidad de realizar **análisis de factores externos** sobre el rendimiento deportivo y la afluencia de público.

In [3]:
# ==============================================================================
# PROYECTO: ENRIQUECIMIENTO CLIMÁTICO DE PARTIDOS (HISTÓRICO)
# DESCRIPCIÓN: Consulta la API de Open-Meteo para obtener temperatura, lluvia y viento
#              en la fecha y hora exacta de cada partido pasado.
# ==============================================================================

# ==============================================================================
# 1. IMPORTACIÓN DE LIBRERÍAS Y CONFIGURACIÓN API
# ==============================================================================
# 'openmeteo_requests': Cliente oficial para conectar con la API del clima.
# 'requests_cache': Para guardar respuestas en memoria y no pedir lo mismo dos veces.
# 'retry_requests': Para reintentar automáticamente si falla la conexión (internet inestable).
# 'urllib3': Para gestionar advertencias de seguridad SSL.

import openmeteo_requests
import requests_cache
from retry_requests import retry
import urllib3

print("============================================")
print("--- INICIO DEL PROCESO DE DATOS CLIMÁTICOS ---")
print("============================================")

# ---------------------------------------------------------
# CONFIGURACIÓN TÉCNICA
# ---------------------------------------------------------
# Desactivamos las advertencias de seguridad SSL (necesario a veces en redes corporativas)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Configuramos el sistema de caché y reintentos para la API
# Esto hace que el script sea robusto: si la API falla un segundo, lo vuelve a intentar 5 veces.
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# URL del servicio de "Archivo" (Histórico) de Open-Meteo
URL_API = "https://archive-api.open-meteo.com/v1/archive"

# ==============================================================================
# 2. CONFIGURACIÓN DE RUTAS (IMPORTANTE: REVISAR ESTO)
# ==============================================================================
# Define aquí dónde está tu archivo con los partidos Y las coordenadas.
# EJEMPLO: Asegúrate de que este CSV tenga columnas 'Latitud', 'Longitud', 'Date' y 'Time'.

RUTA_ARCHIVO_ENTRADA = r"inputs/hop.txt.csv"

# Carpeta donde guardaremos el resultado final con clima
CARPETA_SALIDA = r"outputs"

# ==============================================================================
# 3. CARGA DE DATOS
# ==============================================================================
print("\n-------------------------------------------------------------")
print("--> PASO 1: Cargando archivo de entrada")
print("-------------------------------------------------------------")

if os.path.exists(RUTA_ARCHIVO_ENTRADA):
    df = pd.read_csv(RUTA_ARCHIVO_ENTRADA)
    print(f"   [OK] Archivo cargado: {os.path.basename(RUTA_ARCHIVO_ENTRADA)}")
    print(f"   [INFO] Total de partidos a procesar: {len(df)}")
    
    # Verificación rápida de columnas necesarias
    cols_necesarias = ['Latitud', 'Longitud', 'Date', 'Time']
    # Nota: Si tus columnas se llaman diferente (ej: 'Lat', 'Lon'), cámbialo abajo en el bucle.
    
else:
    print(f"   [ERROR CRÍTICO] No se encuentra el archivo: {RUTA_ARCHIVO_ENTRADA}")
    print("   Por favor, verifica la ruta en la sección 2 del código.")
    exit() # Detiene el programa si no hay archivo

# ==============================================================================
# 4. PREPARACIÓN DE LISTAS DE ALMACENAMIENTO
# ==============================================================================
# Aquí guardaremos los datos que nos vaya dando la API fila por fila
temps = []      # Temperatura (2 metros sobre suelo)
app_temps = []  # Sensación térmica
precips = []    # Precipitación (lluvia)
winds = []      # Velocidad del viento
codes = []      # Código del clima (0=Sol, 61=Lluvia, etc.)

# ==============================================================================
# 5. BUCLE MAESTRO: CONSULTA A LA API (FILA POR FILA)
# ==============================================================================
print("\n-------------------------------------------------------------")
print("--> PASO 2: Conectando con Open-Meteo para cada partido...")
print("-------------------------------------------------------------")

for index, row in df.iterrows():
    try:
        # --- A. VALIDACIÓN PREVIA ---
        # Si falta la latitud o longitud, no podemos pedir clima.
        if pd.isna(row.get('Latitud')) or pd.isna(row.get('Longitud')):
            print(f"   [SALTADO] Fila {index}: Sin coordenadas.")
            temps.append(None); app_temps.append(None); precips.append(None); winds.append(None); codes.append(None)
            continue

        # --- B. EXTRACCIÓN DE DATOS DE LA FILA ---
        lat = row['Latitud']
        lon = row['Longitud']
        date_raw = row['Date']  # Se asume formato DD/MM/YYYY
        time_raw = row['Time']  # Se asume formato HH:MM

        # Convertir fecha de DD/MM/YYYY a YYYY-MM-DD (Formato ISO para la API)
        day, month, year = date_raw.split('/')
        api_date = f"{year}-{month}-{day}"

        # Obtener la hora del partido redondeada (ej: 18:30 -> 18)
        # Esto sirve para buscar el índice en el array de 24 horas que devuelve la API
        match_hour = int(time_raw.split(':')[0])

        # --- C. CONFIGURACIÓN DE LA PETICIÓN ---
        params = {
            "latitude": lat,
            "longitude": lon,
            "start_date": api_date,
            "end_date": api_date,
            "hourly": [
                "temperature_2m",         # Temp real
                "apparent_temperature",   # Sensación
                "precipitation",          # Lluvia
                "wind_speed_10m",         # Viento
                "weather_code"            # Icono/Resumen
            ],
            "timezone": "Europe/Madrid"   # Importante para ajustar la hora
        }

        # --- D. LLAMADA A LA API ---
        # verify=False evita errores de SSL en algunas redes wifi
        responses = openmeteo.weather_api(URL_API, params=params, verify=False)

        # Procesamos la respuesta (La API devuelve un objeto complejo)
        response = responses[0]
        hourly = response.Hourly()

        # --- E. EXTRACCIÓN EXACTA DE LA HORA DEL PARTIDO ---
        # La API devuelve 24 datos (uno por hora del día).
        # Usamos 'match_hour' para coger solo el dato de la hora del pitido inicial.
        t2m = hourly.Variables(0).ValuesAsNumpy()[match_hour]
        att = hourly.Variables(1).ValuesAsNumpy()[match_hour]
        prc = hourly.Variables(2).ValuesAsNumpy()[match_hour]
        wnd = hourly.Variables(3).ValuesAsNumpy()[match_hour]
        wcd = hourly.Variables(4).ValuesAsNumpy()[match_hour]

        # Guardamos en nuestras listas
        temps.append(t2m)
        app_temps.append(att)
        precips.append(prc)
        winds.append(wnd)
        codes.append(wcd)

        # --- F. FEEDBACK EN PANTALLA ---
        # Mostramos progreso cada 20 filas para no saturar la consola
        if (index + 1) % 20 == 0:
            print(f"   > Procesado {index + 1} / {len(df)} partidos... (Fecha: {api_date})")

    except Exception as e:
        # Si algo falla (fecha futura, error de formato, etc.), rellenamos con vacío
        print(f"   [ERROR] Fallo en fila {index} ({date_raw}): {e}")
        temps.append(None)
        app_temps.append(None)
        precips.append(None)
        winds.append(None)
        codes.append(None)

# ==============================================================================
# 6. GUARDADO DE RESULTADOS
# ==============================================================================
print("\n-------------------------------------------------------------")
print("--> PASO 3: Guardando archivo enriquecido")
print("-------------------------------------------------------------")

# Añadimos las listas como nuevas columnas al DataFrame original
df['Temperatura_C'] = temps
df['Sensacion_Termica_C'] = app_temps
df['Precipitacion_mm'] = precips
df['Viento_kmh'] = winds
df['Codigo_Clima'] = codes

# Definimos la ruta de salida
ruta_salida_final = os.path.join(CARPETA_SALIDA, "partidos_con_clima_completo.csv")

# Guardamos el CSV
df.to_csv(ruta_salida_final, index=False, encoding='utf-8-sig')

print(f"✅ PROCESO COMPLETADO EXITOSAMENTE.")
print(f"📂 Archivo guardado en: {ruta_salida_final}")
print("   Contiene nuevas columnas: Temperatura, Lluvia, Viento, etc.")
print("=============================================================")

--- INICIO DEL PROCESO DE DATOS CLIMÁTICOS ---

-------------------------------------------------------------
--> PASO 1: Cargando archivo de entrada
-------------------------------------------------------------
   [OK] Archivo cargado: hop.txt.csv
   [INFO] Total de partidos a procesar: 380

-------------------------------------------------------------
--> PASO 2: Conectando con Open-Meteo para cada partido...
-------------------------------------------------------------
   > Procesado 20 / 380 partidos... (Fecha: 2024-08-25)
   > Procesado 40 / 380 partidos... (Fecha: 2024-09-13)
   > Procesado 60 / 380 partidos... (Fecha: 2024-09-22)
   > Procesado 80 / 380 partidos... (Fecha: 2024-09-30)
   > Procesado 100 / 380 partidos... (Fecha: 2024-10-21)
   > Procesado 120 / 380 partidos... (Fecha: 2024-11-09)
   > Procesado 140 / 380 partidos... (Fecha: 2024-11-30)
   > Procesado 160 / 380 partidos... (Fecha: 2024-12-13)
   > Procesado 180 / 380 partidos... (Fecha: 2024-12-22)
   > Procesado

## HYPE - Medición de Expectación Pública (Google Trends)

### Objetivo
Esta sección cuantifica el **nivel de interés público** que genera cada partido utilizando datos de búsqueda de Google Trends. El objetivo es añadir una métrica que capture la "expectación" o "hype" del enfrentamiento.

### Metodología

#### 1. **Fuente de datos**: Google Trends API (`pytrends`)
- Consulta búsquedas realizadas en **España** (`geo='ES'`)
- Idioma: Español (`hl='es-ES'`)

#### 2. **Término de búsqueda** (MEJORADO):
- **Enfoque anterior**: Buscaba `"Equipo Local vs Equipo Visitante"` (muy específico, pocas búsquedas)
- **Enfoque mejorado**: Busca nombres de **equipos individuales** y calcula el promedio
  - Ejemplo: Para Real Madrid vs Barcelona
    - Busca: `"Real Madrid"` → valor máximo: 85
    - Busca: `"Barcelona"` → valor máximo: 92
    - Hype del partido = (85 + 92) / 2 = **88.5**

#### 3. **Ventana temporal**:
- **7 días** alrededor del partido:
  - 3 días antes
  - Día del partido  
  - 3 días después
- Esto captura el pico de interés antes, durante y después del evento
- Se toma el **valor máximo** de cada equipo en esa ventana

#### 4. **Normalización global** (POST-PROCESAMIENTO):
Problema detectado: Google Trends normaliza valores dentro de cada consulta individual, lo que genera solo valores 0 o 100.

**Solución aplicada**:
- Después de obtener todos los valores brutos, se aplica **normalización Min-Max** sobre todo el dataset:
  - `Hype_normalizado = ((Hype - Hype_min) / (Hype_max - Hype_min)) × 100`
- Esto garantiza una **distribución continua** de valores entre 0 y 100

#### 5. **Protección anti-bloqueo**:
Google Trends tiene límites estrictos de consultas:
- Pausa aleatoria de **5-10 segundos** entre peticiones
- Si detecta error 429 (Too Many Requests), pausa de **60 segundos**
- Proceso completo puede tardar **30-60 minutos** para 380 partidos

### Resultado
Se añade la columna **`Hype_Google_Trends`** al dataset con valores 0-100 (normalizados globalmente) que representan el nivel de expectación pública de cada partido.

**Archivo generado**: `outputs/partidos_completo_con_hype.csv`

### Interpretación
- **Hype alto (>70)**: Partidos muy esperados (Real Madrid, Barcelona, Atlético Madrid, etc.)
- **Hype medio (30-70)**: Partidos con interés moderado
- **Hype bajo (<30)**: Partidos con poco seguimiento mediático

### Ventajas de la metodología mejorada
✅ **Más datos**: Buscar equipos individuales captura más búsquedas que "Local vs Visitante"  
✅ **Distribución continua**: La normalización global elimina el problema de valores binarios (0 o 100)  
✅ **Valores comparables**: Todos los partidos están en la misma escala relativa  
✅ **Refleja popularidad**: El promedio de búsquedas de ambos equipos es un buen proxy del interés del partido

In [6]:
# ==============================================================================
# PROYECTO: MEDICIÓN DE HYPE (GOOGLE TRENDS) - LALIGA 2024-25
# DESCRIPCIÓN: Consulta Google Trends para obtener un índice de interés (0-100)
#              para cada enfrentamiento basándose en búsquedas de equipos.
# MEJORA: Busca equipos individuales + normalización global post-procesamiento
# ==============================================================================

# ==============================================================================
# 1. IMPORTACIÓN DE LIBRERÍAS
# ==============================================================================

import random
import numpy as np
from datetime import datetime, timedelta
from pytrends.request import TrendReq

print("============================================")
print("--- INICIO DEL ANÁLISIS DE HYPE (GOOGLE TRENDS) ---")
print("============================================")

# ==============================================================================
# 2. CONFIGURACIÓN
# ==============================================================================
# Configura la conexión con Google.
# hl='es-ES': Idioma español.
# tz=360: Zona horaria (minutos), aunque no es crítico para datos diarios.
pytrends = TrendReq(hl='es-ES', tz=360)

# RUTAS (AJUSTA ESTO A TU ORDENADOR)
# Usaremos el archivo que ya tiene clima o el de partidos base.
RUTA_ENTRADA = r"outputs/partidos_con_clima_completo.csv"
CARPETA_SALIDA = r"outputs"

# ==============================================================================
# 3. CARGA DE DATOS
# ==============================================================================
print("\n-------------------------------------------------------------")
print("--> PASO 1: Cargando datos de partidos")
print("-------------------------------------------------------------")

if os.path.exists(RUTA_ENTRADA):
    df = pd.read_csv(RUTA_ENTRADA)
    print(f"   [OK] Archivo cargado. Total partidos: {len(df)}")
else:
    print(f"   [ERROR] No se encuentra: {RUTA_ENTRADA}")
    exit()

# ==============================================================================
# 4. FUNCIÓN AUXILIAR: OBTENER HYPE (MEJORADA)
# ==============================================================================
def obtener_hype_google(equipo_local, equipo_visitante, fecha_partido):
    """
    Consulta Google Trends para los nombres de ambos equipos en una ventana de 7 días 
    alrededor del partido. Devuelve el promedio de interés de ambos equipos.
    
    MEJORA: En lugar de buscar "Local vs Visitante" (búsqueda muy específica),
    busca los nombres de cada equipo individualmente para obtener más datos.
    """
    try:
        # 1. Construir la búsqueda: Nombres de equipos individuales
        # Esto captura más búsquedas que "Equipo1 vs Equipo2"
        kw_list = [equipo_local, equipo_visitante]
        
        # 2. Definir ventana de tiempo (3 días antes y 3 días después para capturar el pico)
        # Formato fecha entrada: DD/MM/YYYY -> Convertir a datetime
        fecha_obj = datetime.strptime(fecha_partido, "%d/%m/%Y")
        start_date = (fecha_obj - timedelta(days=3)).strftime("%Y-%m-%d")
        end_date = (fecha_obj + timedelta(days=3)).strftime("%Y-%m-%d")
        
        timeframe = f"{start_date} {end_date}"
        
        # 3. Petición a Google
        # geo='ES': Solo búsquedas en España
        pytrends.build_payload(kw_list, cat=0, timeframe=timeframe, geo='ES', gprop='')
        
        # 4. Obtener datos de interés a lo largo del tiempo
        data = pytrends.interest_over_time()
        
        if not data.empty:
            # Extraer valores máximos para cada equipo
            hype_local = data[equipo_local].max() if equipo_local in data.columns else 0
            hype_visitante = data[equipo_visitante].max() if equipo_visitante in data.columns else 0
            
            # Promedio de ambos equipos como proxy de interés total del partido
            hype_score = (hype_local + hype_visitante) / 2
            return hype_score
        else:
            return 0 # Si no hay datos suficientes
            
    except Exception as e:
        # Google a veces bloquea si hacemos muchas peticiones seguidas (Error 429)
        if "429" in str(e):
            print("   [ALERTA] Bloqueo de Google (Too Many Requests). Esperando...")
            time.sleep(60) # Pausa larga si nos bloquean
        return 0  # Retorna 0 en lugar de None para evitar problemas

# ==============================================================================
# 5. BUCLE PRINCIPAL
# ==============================================================================
print("\n-------------------------------------------------------------")
print("--> PASO 2: Consultando Google Trends (Esto puede tardar...)")
print("-------------------------------------------------------------")

hypes = []

for index, row in df.iterrows():
    local = row['Local']     # O 'HomeTeam' según tu CSV
    visitante = row['Visitante'] # O 'AwayTeam'
    fecha = row['Date']
    
    # Llamada a la función
    hype = obtener_hype_google(local, visitante, fecha)
    hypes.append(hype)
    
    # LOG EN PANTALLA
    # Mostramos barra de progreso
    if (index + 1) % 5 == 0: # Actualiza cada 5 partidos
        print(f"   > {index + 1}/{len(df)} | {local:20s} vs {visitante:20s} -> Hype: {hype:.1f}")
    
    # PAUSA ANTI-BLOQUEO (MUY IMPORTANTE)
    # Google Trends es muy estricto. Necesitamos pausas aleatorias entre peticiones.
    sleep_time = random.randint(5, 10) 
    time.sleep(sleep_time)

# ==============================================================================
# 6. NORMALIZACIÓN GLOBAL (POST-PROCESAMIENTO)
# ==============================================================================
print("\n-------------------------------------------------------------")
print("--> PASO 3: Normalizando valores de Hype globalmente")
print("-------------------------------------------------------------")

if hypes:
    # Convierte a numpy array para facilitar operaciones
    hypes_array = np.array(hypes)
    
    # Normalización Min-Max al rango 0-100 considerando TODOS los partidos
    min_hype = hypes_array.min()
    max_hype = hypes_array.max()
    
    print(f"   Valor mínimo detectado: {min_hype:.2f}")
    print(f"   Valor máximo detectado: {max_hype:.2f}")
    
    if max_hype > min_hype:
        # Aplica normalización Min-Max: (x - min) / (max - min) * 100
        hypes_normalizados = ((hypes_array - min_hype) / (max_hype - min_hype)) * 100
        print(f"   ✅ Normalización aplicada correctamente")
    else:
        # Si todos los valores son iguales, no normalizar
        hypes_normalizados = hypes_array
        print(f"   ⚠️  Todos los valores son iguales, no se aplicó normalización")
    
    df['Hype_Google_Trends'] = hypes_normalizados
else:
    df['Hype_Google_Trends'] = hypes
    print(f"   ⚠️  No hay valores de hype para normalizar")

# ==============================================================================
# 7. GUARDADO FINAL
# ==============================================================================
print("\n-------------------------------------------------------------")
print("--> PASO 4: Guardando resultados")
print("-------------------------------------------------------------")

ruta_final = os.path.join(CARPETA_SALIDA, "partidos_completo_con_hype.csv")
df.to_csv(ruta_final, index=False, encoding='utf-8-sig')

print(f"✅ PROCESO COMPLETADO.")
print(f"📂 Archivo guardado: {ruta_final}")
print(f"   Nueva columna añadida: 'Hype_Google_Trends' (Escala 0-100, normalizada)")
print(f"   Estadísticas finales:")
print(f"      - Media: {df['Hype_Google_Trends'].mean():.2f}")
print(f"      - Mediana: {df['Hype_Google_Trends'].median():.2f}")
print(f"      - Desviación estándar: {df['Hype_Google_Trends'].std():.2f}")
print("=============================================================")

--- INICIO DEL ANÁLISIS DE HYPE (GOOGLE TRENDS) ---

-------------------------------------------------------------
--> PASO 1: Cargando datos de partidos
-------------------------------------------------------------
   [OK] Archivo cargado. Total partidos: 380

-------------------------------------------------------------
--> PASO 2: Consultando Google Trends (Esto puede tardar...)
-------------------------------------------------------------
   > 5/380 | CA Osasuna           vs CD Leganés           -> Hype: 50.0
   > 10/380 | Villarreal CF        vs Atlético de Madrid   -> Hype: 53.0
   > 15/380 | Getafe CF            vs Rayo Vallecano       -> Hype: 51.0
   [ALERTA] Bloqueo de Google (Too Many Requests). Esperando...
   > 20/380 | Atlético de Madrid   vs Girona FC            -> Hype: 71.5
   > 25/380 | Real Valladolid      vs CD Leganés           -> Hype: 50.0
   > 30/380 | FC Barcelona         vs Real Valladolid      -> Hype: 74.5
   [ALERTA] Bloqueo de Google (Too Many Requests). E